# Project 2: Customer‑Support Chatbot for an E-Commerce Store

Welcome! In this project, you'll build a **chatbot** that answers customer service questions about Everstorm Outfitters, an imaginary e-commerce store.

Run each cell in order. Feel free to modify them as you go to better understand each tool and search the web or look online for documentation.

## Learning Objectives  
* **Ingest & chunk** unstructured docs  
* **Embed** chunks and **index** with *FAISS*  
* **Retrieve** context and **craft prompts**  
* **Run** an open‑weight LLM locally with *Ollama*  
* **Build** a Retrieval-Augmented Generation (RAG) chain
* **Package** the chat loop in a minimal **Streamlit** web UI

## Roadmap  
We will build a RAG-based chatbot in **six** steps:

1. **Environment setup**
2. **Data preparation**  
   a. Load source documents  
   b. Chunk the text  
3. **Build a retriever**  
   a. Generate embeddings  
   b. Build a FAISS vector index  
4. **Build a generation engine**. Load the *Gemma3-1B* model through Ollama and run a sanity check.  
5. **Build a RAG**. Connect the system prompt, retriever, and LLM together. 
6. **(Optional) Streamlit UI**. Wrap everything in a simple web app so users can chat with the bot.


## 1 - Environment setup

We use conda to manage our project dependencies and ensure everyone has a consistent setup. Conda is an open-source package and environment manager that makes it easy to install libraries and switch between isolated environments. To learn more about conda, you can read: https://docs.conda.io/en/latest/

Create and activate a clean *conda* environment and install the required packages. If you don't have conda installed, visit https://www.anaconda.com/docs/getting-started/miniconda/main.


Open your terminal, navigate to the project folder where this notebook is located, and run the following commands.

```bash
conda env create -f environment.yml && conda activate rag-chatbot

# (Optional but recommended) Register this environment as a Jupyter kernel
python -m ipykernel install --user --name=rag-chatbot --display-name "rag-chatbot"
```
Once this is done, you can select “rag-chatbot” from the Kernel → Change Kernel menu in Jupyter or VS Code.


> Behind the scenes:
> * Conda reads `environment.yml`, solves all pinned dependencies, and builds an isolated environment named `rag-chatbot`.
> * When it reaches the file’s "pip:" section, Conda automatically invokes pip to install any remaining Python-only packages so the whole stack be available for the project.
> * Registering the kernel makes your new environment visible to Jupyter, so the notebook runs inside the same environment you just created.

Let's import required libraries and print a message if we're not **missing packages**.

In [21]:
# Import standard libraries for file handling and text processing
import os, pathlib, textwrap, glob

# Load documents from various sources (URLs, text files, PDFs)
from langchain_community.document_loaders import UnstructuredURLLoader, TextLoader, PyPDFLoader

# Split long texts into smaller, manageable chunks for embedding
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Vector store to store and retrieve embeddings efficiently using FAISS
from langchain.vectorstores import FAISS

# Generate text embeddings using OpenAI or Hugging Face models
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceEmbeddings, SentenceTransformerEmbeddings

# Use local LLMs (e.g., via Ollama) for response generation
from langchain.llms import Ollama

# Build a retrieval chain that combines a retriever, a prompt, and an LLM
from langchain.chains import ConversationalRetrievalChain

# Create prompts for the RAG system
from langchain.prompts import PromptTemplate

print("✅ Libraries imported! You're good to go!")

✅ Libraries imported! You're good to go!


## 2 - Data preparation
The goal of this step is to turn all reference documents into small chunks of text that a retriever can index and search. These documents typically come from:
* PDF files: local documents such as policies, user manuals, or guides.
* Web pages (HTML): online documentation, blog posts, or help articles.

In this step, we perform two actions:
* **Ingesting**: load every PDF and collect the raw text in a list named `raw_docs`.
* **Chunking**: split each document into small, overlapping chunks so later steps can match a user query to the most relevant passage.

### 2.1 - Ingest source documents

We can use different libraries to load and process PDFs. A quick web search will show several options, each with its own strengths. In this case, we’ll use PyPDFLoader from LangChain, which makes it easy to extract text from PDF files for downstream processing. To learn more about how to use it, refer to: https://python.langchain.com/docs/integrations/document_loaders/pypdfloader/

Use **PyPDFLoader** to load every PDF whose filename matches `data/Everstorm_*.pdf` and collect all pages in a list called `raw_docs`. The content of these PDFs is synthetically generated for educational purposes.

In [5]:
from PyPDF2 import PdfReader
import glob


pdf_paths = glob.glob("data/Everstorm_*.pdf")
raw_docs = []


for p in pdf_paths:
    raw_docs.extend([(page.extract_text() or "").strip() for page in PdfReader(p).pages])


print(f"Loaded {len(raw_docs)} PDF pages from {len(pdf_paths)} files.")

Loaded 4 PDF pages from 4 files.


### (Optional) 2.1 - Load web pages
You can also pull content straight from the web. Various libraries support reading and parsing web pages directly into text, which is useful for building custom knowledge bases. One example is **UnstructuredURLLoader** from LangChain, which can extract readable content from raw HTML pages and return them in a structured format. To learn more, see: https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.url.UnstructuredURLLoader.html

To practice, load each HTML page below and store the results in a list called `raw_docs`. We’ve included a few sample URLs, but you can replace them with any links you prefer.

For robustness, add an offline fallback in case a URL fails. In real projects, we typically cache fetched pages to disk, handle rate limits, and track fetch timestamps so content can be refreshed periodically without relying on live network calls during development. For this project, we don’t have offline HTML copies available, but you can still practice by loading any PDFs from the data/ folder using PyPDFLoader and appending them to raw_docs.

In [6]:
URLS = [
    # --- BigCommerce – shipping & refunds ---
    "https://developer.bigcommerce.com/docs/store-operations/shipping",
    "https://developer.bigcommerce.com/docs/store-operations/orders/refunds",
    # --- Stripe – disputes & chargebacks ---
    # "https://docs.stripe.com/disputes",  
    # --- WooCommerce – REST API reference ---
    # "https://woocommerce.github.io/woocommerce-rest-api-docs/v3.html",
]

try:  
    from langchain_community.document_loaders import UnstructuredURLLoader
    loader = UnstructuredURLLoader(urls=URLS)
    raw_docs = loader.load()

    print(f"Fetched {len(raw_docs)} documents from the web.")
except Exception as e:
    print("⚠️  Web fetch failed, using offline copies:", e)
    raw_docs = []

    
    import glob
    from langchain_community.document_loaders import PyPDFLoader

    for p in glob.glob("data/*.pdf"):
        raw_docs.extend(PyPDFLoader(p).load())
   
    print(f"Loaded {len(raw_docs)} offline documents.")

Fetched 2 documents from the web.


### 2.2 - Chunk the text

Long documents won’t work well directly with most LLMs. They can easily exceed the model’s context window, making it impossible for the model to read or reason over the full text at once. Even if they fit, processing long inputs can be inefficient and lead to weaker retrieval results.

To handle this, we split large documents into smaller, overlapping chunks. Several libraries can help with text splitting, each designed to preserve structure or balance chunk size. A popular choice is `RecursiveCharacterTextSplitter` from LangChain, which splits text intelligently while keeping paragraph or sentence boundaries intact. To familiarize youself with the library, visit: https://python.langchain.com/api_reference/text_splitters/character/langchain_text_splitters.character.RecursiveCharacterTextSplitter.html

In this project, we’ll split each document into chunks of roughly 300 tokens with a 30-token overlap using `RecursiveCharacterTextSplitter`. This overlap helps maintain continuity across chunks while ensuring each piece stays small enough for embedding and retrieval.

In [8]:
chunks = []

from langchain_text_splitters import RecursiveCharacterTextSplitter
chunks = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=300, chunk_overlap=30).split_documents(raw_docs)

print(f"✅ {len(chunks)} chunks ready for embedding")

✅ 21 chunks ready for embedding


## 3 -Build a retriever

A *retriever* lets the RAG pipeline efficiently look up small, relevant pieces of context at query‑time. This step has two parts:
1. **Load a model to generate embeddings**: convert each text chunk from the reference documents into a fixed‑length vector that captures its semantic meaning.  
2. **Build vector database**: store these embeddings in a vector database.


### 3.1 - Load a model to generate embeddings

The goal of this step is to convert each document chunk into a numerical vector (an embedding) that captures its semantic meaning. These embeddings allow our retriever to find and compare similar pieces of text efficiently.

There are models trained specifically for this purpose, called embedding models. One popular example is OpenAI’s `text-embedding-3-small`, which produces high-quality embeddings that work well for retrieval and semantic search.

If you prefer running everything locally, you can use smaller open-source models such as `gte-small` (77 M parameters). These local models load quickly, don’t require internet access, and are ideal for experimentation or environments without API access. However, they’re typically less powerful than hosted models.

Alternatively, you can connect to an API service to access stronger models like OpenAI’s. These require setting an API key (for example, OPENAI_API_KEY) in your environment. OpenAI allows you to create a free account and sometimes offers limited trial credits for new users, but ongoing access requires a billing setup. 

In this exercise, we’ll stick to the smaller gte-small model for simplicity and reproducibility. We'll use our imported `SentenceTransformerEmbeddings` library to load the model and use it to embed queries. To learn more about lagnchain's embedding support, visit: https://python.langchain.com/docs/integrations/text_embedding/

In [9]:
embedding_vector = []

# Embed the sentence "Hello world! and store it in an embedding_vector.

from langchain_community.embeddings import SentenceTransformerEmbeddings

# Initialize the embedding model
embedder = SentenceTransformerEmbeddings(model_name="thenlper/gte-small")

# Embed the sentence
embedding_vector = embedder.embed_query("Hello world!")

print(len(embedding_vector))

C:\Users\cmiyachi\AppData\Local\Temp\ipykernel_37996\1153428590.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedder = SentenceTransformerEmbeddings(model_name="thenlper/gte-small")
c:\Users\cmiyachi\AppData\Local\miniconda3\envs\rag-chatbot\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\cmiyachi\.cache\huggingface\hub\models--thenlper--gte-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by sett

384


### 3.2 - Build a vector database

Once we have embeddings, we need a way to store and search them efficiently. A simple list wouldn’t scale well, especially when we have thousands of chunks and need to quickly find the most relevant ones.

To solve this, we use **FAISS**, an open-source similarity search library developed by Meta. FAISS is optimized for fast nearest-neighbor search in high-dimensional spaces, making it ideal for tasks like semantic retrieval and recommendation. It’s strongly encouraged to visit their quickstart guide to understand how FAISS works and how to use it effectively: https://github.com/facebookresearch/faiss/wiki/getting-started

In this step, we’ll feed all our document embeddings into FAISS, which builds an in-memory vector index. This index allows us to efficiently query for the *k* most similar chunks to any given question.

During inference, we’ll use this index to retrieve the top-k relevant chunks and pass them to the LLM as context, enabling it to answer questions grounded in our documents.



In [10]:
# Expected steps:
    # 1. Build the FAISS index from the list of document chunks and their embeddings.
    # 2. Create a retriever object with a suitable k value (e.g., 8).
    # 3. Save the vector store locally (e.g., under "faiss_index").
    # 4. Print a short confirmation showing how many embeddings were stored.

from langchain_community.vectorstores import FAISS

# 1) Build FAISS index from chunks
vectordb = FAISS.from_documents(chunks, embedder)

# 2) Create a retriever with k=8
retriever = vectordb.as_retriever(search_kwargs={"k": 8})

# 3) Save the vector store locally
vectordb.save_local("faiss_index")

# 4) Print confirmation
print("✅ Vector store with", vectordb.index.ntotal, "embeddings")

✅ Vector store with 21 embeddings


## 4 - Build the generation engine
At the core of any RAG system lies an **LLM**. The retriever finds relevant information, and the LLM uses that information to generate coherent, context-aware responses.

In this project, we’ll use **Gemma 3* (1B), a small but capable open-weight model, and run it entirely on your local machine using Ollama. This means you won’t need API keys or internet access to generate responses once the model is downloaded.

**What is Ollama?**

Ollama is a lightweight runtime for managing and serving open-weight LLMs locally. It provides:
* A simple REST API running at localhost:11434, so your code can interact with the model via standard HTTP calls.
* A model registry and command-line tool** to pull, run, and manage models easily.
* Support for a wide variety of models (e.g., Gemma, Llama, Mistral, Phi, etc.), making it ideal for experimentation.

To learn more about Ollama, visit: https://github.com/ollama/ollama. You can browse all supported models and their sizes here: https://ollama.com/library


### 4.1 - Install `ollama` and serve `gemma3`

Follow these steps to set up Ollama and start the model server:

**1 - Install**
```bash
# macOS (Homebrew)
brew install ollama
# Linux
curl -fsSL https://ollama.com/install.sh | sh
```

If you’re on Windows, install using the official installer from https://ollama.com/download.

**2 - Start the Ollama server (keep this terminal open)**
```bash
ollama serve
```
This command launches a local server at http://localhost:11434, which will stay running in the background.


**3 - Pull the Gemma mode (or the model of your choice) in a new terminal**
```bash
ollama pull gemma3:1b
```

This downloads the 1B version of Gemma 3, a compact model suitable for running on most modern laptops. Once downloaded, Ollama will automatically handle model loading and caching.


After this setup, your system is ready to generate responses locally using the Gemma model through the Ollama API.


### 4.2 - Test an LLM with a random prompt (Sanity check)


In [17]:
# Expected steps:
    # 1. Initialize the model (for example, gemma3:1b) with a low temperature such as 0.1 for more factual outputs.
    # 2. Use llm.invoke() with a short test prompt and print the response to verify that the model runs successfully.

from langchain_community.llms import Ollama

# 1) Initialize the model (gemma3:1b) with low temperature for factual outputs
llm = Ollama(model="deepseek-r1:8b", temperature=0.1)

# 2) Test the model with a short prompt
response = llm.invoke("Hello! Can you summarize what you are?")
print(response)

Hello! 😊 I'm DeepSeek-R1, your friendly AI assistant, designed to help you with all sorts of questions, ideas, and tasks. I can understand and generate human-like text, answer questions, explain concepts, help with writing, summarize information, analyze data, and so much more!

I was trained on a wide range of texts up to June 2024, so I know a lot about science, history, technology, art, everyday life, and more. While I don't have real-time browsing turned on by default, I can still help with most topics unless they're super recent.

Think of me as your knowledgeable companion—always ready to chat, learn, and assist. 🧠💬

Is there anything you'd like to know or talk about?


## Build a RAG

### 5.1 - Define a system prompt

At this stage, we need to tell the model how to behave when generating answers. The **system prompt** acts as the model’s rulebook. It should clearly instruct the model to answer only using the retrieved context and to admit when it doesn’t know the answer. This helps prevent hallucination and keeps the responses grounded in the provided documents.

In general, a good RAG prompt emphasizes three things: stay within context, stay factual, and stay concise. This is important because RAG works by grounding the LLM in retrieved text. If the prompt is vague, the model may invent details. A precise system prompt reduces hallucinations and keeps answers aligned with your corpus.

In [18]:
SYSTEM_TEMPLATE = """
You are a **Customer Support Chatbot**. Use only the information in CONTEXT to answer.
If the answer is not in CONTEXT, respond with “I'm not sure from the docs.”

Rules:
1) Use ONLY the provided <context> to answer.
2) If the answer is not in the context, say: "I don't know based on the retrieved documents."
3) Be concise and accurate. Prefer quoting key phrases from the context.
4) When possible, cite sources as [source: source] using the metadata.

CONTEXT:
{context}

USER:
{question}
"""


from langchain.prompts import PromptTemplate

# Create the system prompt template
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=SYSTEM_TEMPLATE
)

# Example usage:
formatted_prompt = prompt.format(context="Sample context from docs", question="How do I request a refund?")
print(formatted_prompt)



You are a **Customer Support Chatbot**. Use only the information in CONTEXT to answer.
If the answer is not in CONTEXT, respond with “I'm not sure from the docs.”

Rules:
1) Use ONLY the provided <context> to answer.
2) If the answer is not in the context, say: "I don't know based on the retrieved documents."
3) Be concise and accurate. Prefer quoting key phrases from the context.
4) When possible, cite sources as [source: source] using the metadata.

CONTEXT:
Sample context from docs

USER:
How do I request a refund?



### 5.2 Create a RAG chain
Now that we have a retriever, a prompt, and a language model, we can connect them into a single RAG pipeline. The retriever finds the most relevant chunks from our vector index, the prompt injects those chunks into the system message, and the LLM uses that context to produce the final answer. (retriever → prompt → model)

This connection is handled through LangChain’s `ConversationalRetrievalChain`, which combines retrieval and generation. To familiarize yourself with the library, visit: https://python.langchain.com/api_reference/langchain/chains/langchain.chains.conversational_retrieval.base.ConversationalRetrievalChain.html

In [19]:
# Expected steps:
    # 1. Create a PromptTemplate that uses the SYSTEM_TEMPLATE you defined earlier, with input variables for "context" and "question".
    # 2. Initialize your LLM using Ollama with the gemma3:1b model and a low temperature (e.g., 0.1) for reliable, grounded responses.
    # 3. Build a ConversationalRetrievalChain by combining the LLM, the retriever, and your custom prompt and name it "chain".

from langchain.prompts import PromptTemplate
from langchain_community.llms import Ollama
from langchain.chains import ConversationalRetrievalChain

prompt = PromptTemplate(input_variables=["context", "question"], template=SYSTEM_TEMPLATE)
# llm = Ollama(model="gemma3:1b", temperature=0.1)
llm = Ollama(model="deepseek-r1:8b", temperature=0.1)
chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, combine_docs_chain_kwargs={"prompt": prompt})

result = chain.invoke({"question": "How do I process refunds?", "chat_history": []})
print(result["answer"])

To process refunds, use the **Order V3 Refund capabilities**. Refunds require settled payments and compatible payment gateways. The process involves two API requests:

1. **Create a Refund Quote**: Use the endpoint `/v3/orders/{order_id}/payment_actions/refund_quotes` to calculate amounts and get payment methods.
2. **Create the Refund**: Use the endpoint `/v3/orders/{order_id}/payment_actions/refunds` with the provider_id, amount, and items from the quote.

You can refund specific items (PRODUCT, SHIPPING, etc.) or create **order-level refunds** by setting `item_type` to `ORDER`. For more details, see the **Order Refunds** article [source: Order Refunds] and the **Order V3 Reference** [source: Order V3 Reference].


When you ask a question, the retriever pulls the top few relevant text chunks, the model reads them through the system prompt, and then it generates an answer based on that context.

This structure makes the system transparent and easy to debug. You can inspect what text was retrieved, tune parameters like k, and experiment with different prompts to see how they affect the output quality.


### 5.3 - Validate the RAG chain

We run a few questions to make sure everything behaves as expecte. Experiment by adding you own questions.

In [20]:
test_questions = [
    "If I'm not happy with my purchase, what is your refund policy and how do I start a return?",
    "How long will delivery take for a standard order, and where can I track my package once it ships?",
    "What's the quickest way to contact your support team, and what are your operating hours?",
]

# Expected steps:
    # 1. Initialize an empty chat_history list.
    # 2. Loop through test_questions, pass each question and the current chat history to the chain, and append the new answer.
    # 3. Print each question and the LLM's response to verify it’s working correctly.

# 1) Initialize empty chat history
chat_history = []

# 2) Loop through test questions
for q in test_questions:
    result = chain.invoke({"question": q, "chat_history": chat_history})
    answer = result["answer"]
    
    # Append the interaction to chat history
    chat_history.append((q, answer))
    
    # 3) Print question and response
    print(f"Q: {q}\nA: {answer}\n{'-'*60}")

Q: If I'm not happy with my purchase, what is your refund policy and how do I start a return?
A: I'm not sure from the docs.
------------------------------------------------------------
Q: How long will delivery take for a standard order, and where can I track my package once it ships?
A: I'm not sure from the docs.
------------------------------------------------------------
Q: What's the quickest way to contact your support team, and what are your operating hours?
A: I'm not sure from the docs.
------------------------------------------------------------


### 6 - Build the Streamlit UI (optional)

The goal here is to create a tiny demo so you can interact with your RAG system. The focus is not on UI design. We will build a very small interface only to demonstrate the end-to-end flow.

There are many ways to make a UI. Some frameworks are powerful but take longer to set up, while others are simple and good for quick experiments. Streamlit is a common choice for fast prototyping because it lets you make a usable interface with only a few lines of Python. If you want to learn the basics, see the Streamlit Quickstart: https://docs.streamlit.io/deploy/streamlit-community-cloud/get-started/quickstart

This step is optional. If it is not useful for your work, you can skip it. We will also complete this part together during the live session.

In this cell, we write a minimal **`app.py`** that starts a simple chat UI and calls your RAG chain.

In [ ]:
# app.py
# Minimal Streamlit chat app for your RAG pipeline.
# Prereqs (example):
#   pip install streamlit langchain langchain-community sentence-transformers faiss-cpu pypdf
#   ollama pull gemma3:1b

import os
import streamlit as st

from langchain.prompts import PromptTemplate
from langchain_community.llms import Ollama
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain.chains import ConversationalRetrievalChain

# -----------------------------
# Config & constants
# -----------------------------
DEFAULT_INDEX_DIR = os.environ.get("FAISS_INDEX_DIR", "faiss_index")
DEFAULT_MODEL = os.environ.get("OLLAMA_MODEL", "deepseek-r1:8b")

SYSTEM_TEMPLATE = """
You are a **Customer Support Chatbot**. Use only the information in CONTEXT to answer.
If the answer is not in CONTEXT, respond with “I'm not sure from the docs.”

Rules:
1) Use ONLY the provided <context> to answer.
2) If the answer is not in the context, say: "I don't know based on the retrieved documents."
3) Be concise and accurate. Prefer quoting key phrases from the context.
4) When possible, cite sources as [source: source] using the metadata.

CONTEXT:
{context}

USER:
{question}
""".strip()


# -----------------------------
# Helpers
# -----------------------------
@st.cache_resource(show_spinner=True)
def load_vectordb(index_dir: str, embed_model_name: str = "thenlper/gte-small"):
    """Load a saved FAISS index and return a retriever."""
    embedder = SentenceTransformerEmbeddings(model_name=embed_model_name)
    # allow_dangerous_deserialization=True is needed for FAISS pickle
    vectordb = FAISS.load_local(index_dir, embedder, allow_dangerous_deserialization=True)
    retriever = vectordb.as_retriever(search_kwargs={"k": 8})
    return vectordb, retriever

@st.cache_resource(show_spinner=True)
def build_chain(retriever, model_name: str = DEFAULT_MODEL, temperature: float = 0.1):
    """Create the LLM and ConversationalRetrievalChain with a custom prompt."""
    prompt = PromptTemplate(input_variables=["context", "question"], template=SYSTEM_TEMPLATE)
    llm = Ollama(model=model_name, temperature=temperature)
    chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        combine_docs_chain_kwargs={"prompt": prompt},
    )
    return chain

def render_sources(docs):
    """Render source citations under the answer."""
    if not docs:
        return
    st.caption("**Sources:**")
    for i, d in enumerate(docs, 1):
        meta = d.metadata or {}
        src = meta.get("source", "unknown")
        page = meta.get("page", None)
        if page is not None:
            st.caption(f"{i}. [source: {src}] (page {page})")
        else:
            st.caption(f"{i}. [source: {src}]")

def init_session_state():
    if "chat_history" not in st.session_state:
        st.session_state.chat_history = []  # list of (user, assistant) tuples
    if "chain" not in st.session_state:
        st.session_state.chain = None


# -----------------------------
# UI
# -----------------------------
st.set_page_config(page_title="RAG Demo", page_icon="💬", layout="centered")
st.title("💬 RAG Chat Demo")
st.write("Ask questions grounded in your indexed documents.")

with st.sidebar:
    st.header("⚙️ Settings")
    index_dir = st.text_input("FAISS index directory", value=DEFAULT_INDEX_DIR)
    model_name = st.text_input("Ollama model", value=DEFAULT_MODEL)
    temperature = st.slider("Temperature", min_value=0.0, max_value=1.0, value=0.1, step=0.05)
    if st.button("Clear chat"):
        st.session_state.chat_history = []
        st.toast("Chat history cleared")

init_session_state()

# Load vector store & build chain (cached)
try:
    vectordb, retriever = load_vectordb(index_dir)
    st.session_state.chain = build_chain(retriever, model_name=model_name, temperature=temperature)
except Exception as e:
    st.error(f"Failed to load FAISS index or build chain: {e}")
    st.stop()

# Render previous messages
for user_msg, assistant_msg in st.session_state.chat_history:
    with st.chat_message("user"):
        st.markdown(user_msg)
    with st.chat_message("assistant"):
        st.markdown(assistant_msg)

# Chat input
user_input = st.chat_input("Type your question…")
if user_input:
    with st.chat_message("user"):
        st.markdown(user_input)

    # Invoke RAG chain
    try:
        result = st.session_state.chain.invoke(
            {"question": user_input, "chat_history": st.session_state.chat_history}
        )
        answer = result.get("answer", "")
        source_docs = result.get("source_documents", [])

        with st.chat_message("assistant"):
            st.markdown(answer)
            render_sources(source_docs)

        # Update chat history
        st.session_state.chat_history.append((user_input, answer))

    except Exception as e:
        with st.chat_message("assistant"):
            st.error(f"Error generating answer: {e}")

2025-11-16 18:13:55.519 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-16 18:13:55.519 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-16 18:13:56.028 
  command:

    streamlit run c:\Users\cmiyachi\AppData\Local\miniconda3\envs\rag-chatbot\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-11-16 18:13:56.030 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-16 18:13:56.030 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-16 18:13:56.032 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-16 18:13:56.033 Thread 'MainThread': missing ScriptRunContext! Th

Run `streamlit run app.py` from your terminal.

## 🎉 Congratulations!

You’ve just built, tested, and demoed a fully working **customer-support chatbot**.  
In one project you:

* **Prepared policy docs**: loaded and chunked them for fast retrieval.  
* **Built a vector store**: created a FAISS index with text embeddings.  
* **Plugged in an LLM**: wrapped Gemma3 with LangChain and a prompt-aware RAG chain.  
* **Validated end-to-end**: answered refund, shipping, and contact questions with confidence.  

Swap in new documents, tweak the prompt, and your store’s customers get instant, accurate answers.

👏 **Great job!** Take a moment to celebrate. The skills you used here power most RAG-based chatbots you see everywhere.
